In [36]:
import gym
import torch
import myrl 
from myrl.buffers import ReplayBuffer, PrioritizedReplayBuffer
from myrl.visualizer import showit
from myrl.utils import ExperimentWriter, get_batch_obs, get_batch_a, Discrete2Continuous
from myrl.value_functions import polyak, DQN
from myrl.policies import RandomPolicy
from myrl.environments import Envs

envname = 'LunarLander-v2'
envname = 'CartPole-v0'
env = gym.make(envname)
envs = Envs(envname, 1)


wll = ExperimentWriter('tb/' + envname + " -")

In [38]:
rbuff = ReplayBuffer(10000)

In [45]:
print(env.action_space, env.observation_space)
adim = 2 #env.action_space.shape[0]
sdim = env.observation_space.shape[0]
dqn1 = DQN([sdim, 32, 32, adim])
dqn2 = DQN([sdim, 32, 32, adim])

import copy
tdqn1 = copy.deepcopy(dqn1)
tdqn2 = copy.deepcopy(dqn2)

dqn1

Discrete(2) Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)


DQN(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=2, bias=True)
  )
)

In [46]:
opt = torch.optim.Adam(list(dqn1.parameters())+list(dqn2.parameters()), lr=1e-3)

In [48]:
#wll.new()
writer = wll.writer

bsize = 32
warmup = 400 
gamma = 1
collector = lambda obs: dqn1.act(obs, epsilon=0.15)
shower    = lambda obs: dqn1.act(obs, epsilon=0)
random_policy = RandomPolicy(env).act



for ep in range(400, 1000):
    obs = env.reset()
    if ep%1==0:
        pi = collector if len(rbuff)>bsize*5 else random_policy
        oldobs, _, r, obs, d, _, _, a = envs.rollout(pi)
        rbuff.add(oldobs, a, r, obs, d)
        writer.add_scalar('dqn/reward', r.sum(), ep)
        # writer.add_scalar('a/reward', rss, ep)
    # print(r.sum())
    if bsize*10 > len(rbuff):
        continue

    for i in range(15):
        oldobs, a, r, obs, done = rbuff.get(bsize)
        for opt_step in range(4):
            import random
            if random.uniform(0, 1) > 0.5:
                max_action = dqn1.get_action(obs)
                target = dqn2.get_q(obs, max_action)
            else:
                max_action = dqn2.get_action(obs)
                target = dqn1.get_q(obs, max_action)

            target = target*gamma*(1-done) + r            
            calc1 = dqn1.get_q(oldobs, a)
            calc2 = dqn2.get_q(oldobs, a)
            l2 = ((target - calc2)**2).mean()
            l1 = ((target - calc1)**2).mean()
            loss = l1 + l2
            opt.zero_grad()
            loss.backward()
            opt.step()

    writer.add_scalar('a/dqn1', l1.item(), ep)
    writer.add_scalar('a/dqn2', l2.item(), ep)
    
    polyak(dqn1, tdqn1, 1-1/50)
    polyak(dqn2, tdqn2, 1-1/50)
    
    if ep%10==0:
        print(ep, loss.item(), r.sum())

400 0.6886881589889526 tensor(32.)
410 0.43063944578170776 tensor(32.)
420 0.36125636100769043 tensor(32.)
430 0.5335972309112549 tensor(32.)
440 0.5002211332321167 tensor(32.)
450 0.4755089581012726 tensor(32.)
460 0.4661303162574768 tensor(32.)
470 0.3455895483493805 tensor(32.)
480 0.5163539052009583 tensor(32.)
490 0.5611607432365417 tensor(32.)
500 0.3459252715110779 tensor(32.)
510 0.458261638879776 tensor(32.)
520 0.4232293963432312 tensor(32.)
530 0.6557198762893677 tensor(32.)
540 0.45939815044403076 tensor(32.)
550 0.6972138285636902 tensor(32.)
560 0.8536415696144104 tensor(32.)
570 0.7665558457374573 tensor(32.)
580 0.3403170108795166 tensor(32.)
590 0.49962857365608215 tensor(32.)
600 0.6635268926620483 tensor(32.)
610 0.14883294701576233 tensor(32.)
620 0.5155174732208252 tensor(32.)
630 0.6763342618942261 tensor(32.)
640 0.5395363569259644 tensor(32.)
650 0.4958474636077881 tensor(32.)
660 1.1010005474090576 tensor(32.)
670 0.975433349609375 tensor(32.)
680 0.62487757205